# Scalable Data Mining: Assignment 1

## Movie recommendation using SVD

In this assignment, you will perform the following tasks.

1. Load the MovieLens data and Convert the data into a matrix (for now, let it be dense)
2. (Optional: if the data is too big to handle, choose a subset of the ratings)
3. Leave out some non zero entries as a test set
4. Perform normalization 
5. Perform SVD
6. Compute the low rank ratings matrix according to the basic latent factor model
7. Test performance against the test data that is left out

First, download the data from: https://grouplens.org/datasets/movielens/

There are two versions recommended for education and development: ml-latest and ml-latest-small. It is recommended to download the small version and complete this assignment. However, if you can manage the larger version, you are welcome to do so. The ratings.csv file is the one you need to use for this assignment. 

### What to submit: 

Please submit your jupyter notebook file and a brief readme. You may mention any challenges you encountered, or any comments about the experiments you performed (for example, your choice of k for SVD). Please name both your files with your roll number. For example, if your roll number is AI007, your files should be named `ai007.ipynb` and `ai007.txt` respectively.

### 1. Load the dataset and convert the data into a matrix

Load the ratings.csv file and convert the data into a matrix R. Depending on how you load it, your movies may be rows or columns (and users vice versa). Mention that in the comment. 

In [1]:
# Write code here (placeholder), with the required imports. Finally, your matrix should be called A.
import pandas as pd
import numpy as np
ratings_df=pd.read_csv('ratings.csv')
#ratings_df.head()
# Mention: for my matrix, the movies are columns and the users are rows
ratings_df_pivot = ratings_df.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
#ratings_df_pivot.head()
# R =
R = ratings_df_pivot.values
R
#print(np.count_nonzero(~np.isnan(R[6])))

array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

### 2. Optional: Reduce the size of the matrix

If you failed trying to run SVD with your matrix due to computational resources, reduce the size of the matrix and call the new matrix R. If you did not need to reduce the size of R, then simply set skip this step.

In [2]:
# R = 

### 3. Set your test set apart

Randomly choose about 2000 non-zero entries in R (2000 known ratings given by users to certain items) and create the test matrix B. Define A to be the rest of the non-zero entries. Essentially, A + B should be same as R. After the next few lines of code, you should have two matrices A and B. 

Note: A and B should have no common non-zero entry. The task here is to predict the zero (unknown) entries in A and test them against the available non-zero (known) entries in B.

a) For 2000 non-zero samples, randomly selected 100 users who had given some ratings to a sample of randomly selected 20 movies each.

In [3]:
def train_test_split(ratings):
    test = np.zeros(ratings.shape)
    test_copy=np.zeros((100,20))
    train = ratings.copy()
    t=[]
    c=0
   
    n = ratings.shape[0]  # for 2 random indices
    user=np.empty((0))
    for i in np.arange(n):
        user=np.append(user,i)
    index = np.random.choice(user, n, replace=False)
    index=index.astype('int')
    for user in index:
        if(c==100):
            break
        test_ratings = np.random.choice(ratings[user, :].nonzero()[0], 
                                        size=20, 
                                        replace=False)
        #print(test_ratings)
        train[user, test_ratings] = 0.
        c=c+1
        #print(c)
        test[user, test_ratings] = ratings[user, test_ratings]
        t.append(test[user, test_ratings])
    # Test and training are truly disjoint
    assert(np.all((train * test) == 0)) 
    return train, test
# B = 
# A = 
A,B=train_test_split(R)
print(A)
# print(B)

[[4.  0.  4.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [2.5 2.  2.  ... 0.  0.  0. ]
 [3.  0.  0.  ... 0.  0.  0. ]
 [5.  0.  0.  ... 0.  0.  0. ]]


### 4. Normalize and center 

Please note that in A, B or R, the zero entries do not mean the corresponding ratings are zero. They are just unknown. Hence, you need to impute the missing (zero now) entries of A by the average of the known entries (row or column depending on how you have loaded your matrix) for each item. Call this matrix A1.

Then, subtract the average rating given by each user from the corresponding row (or column). Call this matrix A2.

a) Here row mean is calculated i.e. average rating given by user to movies.

In [4]:
A[A == 0] = np.NaN
A1=A.copy()
print(A1)
row_mean = np.nanmean(A1, axis=1)
inds = np.where(np.isnan(A1))
A1[inds] = np.take(row_mean, inds[0])
A1[np.isnan(A1)] = 0.
print(A1)
A2=A1-row_mean.reshape(-1,1)
print(A2)
A2[np.isnan(A2)] = 0.
A2=A2.astype(np.float64)
# A1 = 
# A2 = 
row_mean[np.isnan(row_mean)] = 0.

[[4.  nan 4.  ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [2.5 2.  2.  ... nan nan nan]
 [3.  nan nan ... nan nan nan]
 [5.  nan nan ... nan nan nan]]


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.


[[4.         4.36637931 4.         ... 4.36637931 4.36637931 4.36637931]
 [3.94827586 3.94827586 3.94827586 ... 3.94827586 3.94827586 3.94827586]
 [2.43589744 2.43589744 2.43589744 ... 2.43589744 2.43589744 2.43589744]
 ...
 [2.5        2.         2.         ... 3.13417569 3.13417569 3.13417569]
 [3.         3.27027027 3.27027027 ... 3.27027027 3.27027027 3.27027027]
 [5.         3.68855607 3.68855607 ... 3.68855607 3.68855607 3.68855607]]
[[-0.36637931  0.         -0.36637931 ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [-0.63417569 -1.13417569 -1.13417569 ...  0.          0.
   0.        ]
 [-0.27027027  0.          0.         ...  0.          0.
   0.        ]
 [ 1.31144393  0.          0.         ...  0.          0.
   0.        ]]


### 5. Compute SVD 

Compute SVD of the matrix A2.

In [5]:
# Use your choice of variables for U, Sigma and V_transpose
u,s,vt=np.linalg.svd(A2,full_matrices=False)
rank=np.linalg.matrix_rank(A2)
# Compute SVD of A2

### 6. Compute the low rank approximation 

Compute the low rank approximation of A2. The choice of k is up to you, but we would encourage you to try out a few different k and see which works better. Mention your experience in the readme report.

In [6]:
# Get Ak matrix of rank k.
def calc_A_k(u,s,vt,r):
    u=u[:,:r]
    vt=vt[:r,:]
    s=s[:r]
    s=np.diag(s)
    A_k=np.dot(np.dot(u,s),vt)
    return A_k

<ul><li>For low rank k approximation used Eckart-Young Theorem.</li>
<li>Eckart-Young Theorem states that for any Ak,argmin(k<=r) where r is the rank of matrix A then the 2 norm error i.e. ||A-Ak|| is equal to the first singular value(k+1) that we did not use.</li>
<li>From the above,the low rank k value approximation for A2 matrix was obtained by storing the absolute minimum difference between ||A-Ak|| and singular_value(k+1) i.e. converges to 0 in a list and getting the minimum argument k corresponding to it.</li>
<li>The minimum argument k is used to reconstruct the Ak matrix and RMSE error is obtained correspondingly.</li></ul>

In [7]:
# Calculate low rank k using Eckart-Young Theorem.Computationally the snippet will take some time to run due to matrix dimensionality.
approxk=dict()
for i in range(0,s.size-1):
    z=calc_A_k(u,s,vt,i+1)
    v=np.linalg.norm(A2-z,2)
    approxk[i]=abs(v-s[i+1])
#print(approxk)

In [8]:
# Set your k
minval = min(approxk.values())
result = list(filter(lambda x: approxk[x]==minval, approxk))
k = min(result)
print(k+1)
# Compute low rank Ak
# Ak =
Ak=calc_A_k(u,s,vt,k+1)

31


### 7. Test the performance

Compute the root-mean-squared-error (RMSE) against the known ratings to test how the method performed, using A1 (which predicts ratings just by average known ratings) and Ak against B.

The RMSE is the square root of the average squared errors between the known ratings and the predicted ratings. For our set up, consider only the pairs of (user,item) pairs (i,j) for which B[i,j] > 0. Then compute the square root of the average squared difference between A2 and B (also Ak and B) only for this set of pairs (i,j). 

$$RMSE(A2, B) = \sqrt{\frac{\sum_{i,j: B_{ij} > 0}(A2_{ij} - B_{ij})^2}{\sum_{i,j: B_{ij} > 0} 1}}.$$

$$RMSE(Ak, B) = \sqrt{\frac{\sum_{i,j: B_{ij} > 0}(Ak_{ij} - B_{ij})^2}{\sum_{i,j: B_{ij} > 0} 1}}.$$

Based on positive ratings of B the indices are calculated which are used correspondingly in A2 and Ak respectively for performing RMSE calculations.

In [9]:
# Write your code here (several lines)
# At the end of the code, your must have two variables with the RMSEs (will not be 0.0), as given below
RMSE_A2 = 0.0
RMSE_Ak = 0.0
result = np.where(B > 0)
def compute_RMSE(X):
    n=0
    rsum=0.0
    for i in result[0]:
        rsum=rsum+(X[i][result[1][i]]-B[i][result[1][i]])**2
        n=n+1
    rmse=np.sqrt(rsum/n)
    return rmse
RMSE_A2=compute_RMSE(A2)
RMSE_Ak=compute_RMSE(Ak)
print("RMSE with A2 is %f." %RMSE_A2)
print("RMSE with Ak is %f." %RMSE_Ak)

RMSE with A2 is 0.658350.
RMSE with Ak is 0.613216.
